# Month by month opioid data analysis for Texas

In [1]:
import pandas as pd
import numpy as np
import gzip
import os
import glob
import datetime

## Analysis begins here

In [21]:
# Don't run this part!!! The raw data is on Shining's laptop locally

zipfiles = "/Users/yangshining/Desktop/2021 fall @duke/IDS 720/mid-sem project/data/opioid_shipment/texas/*.gz"

filelist = glob.glob(zipfiles)

select_data = []

for gzfile in filelist:
    raw = pd.read_csv(gzfile, iterator= True, chunksize = 10000, compression='gzip', error_bad_lines=False)


    for df in raw:

        df["TRANSACTION_DATE1"] = pd.to_datetime(df["TRANSACTION_DATE"], format='%m%d%Y') # Convert transaction data to standardized date time

        df["year"] = pd.DatetimeIndex(df["TRANSACTION_DATE1"]).year # Extract year

        df["month"] = pd.DatetimeIndex(df["TRANSACTION_DATE1"]).month # Extract month

        df["year_month"] = df['TRANSACTION_DATE1'].dt.strftime('%Y-%m') # Create year-month column

        df = df[["BUYER_COUNTY", "BUYER_STATE", "year", "month", "year_month", "MME"]] # Subset the columns that we need

        select_data.append(df) # Append the chunks to the list



whole_data = pd.concat(select_data) # Concatenate data

grouped1 = whole_data.groupby(["BUYER_STATE", "BUYER_COUNTY", "year", "month"], as_index=False).sum("MME")
# grouped1: groupby year and month

grouped2 = whole_data.groupby(["BUYER_STATE", "BUYER_COUNTY", "year_month"], as_index=False).sum("MME")
#grouped2: groupby year_month column


# Test if the data is correctly loaded

#print(len(grouped))

grouped1["BUYER_STATE"].value_counts()

/Users/yangshining/opt/miniconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


TX    24042
OK     8292
LA     6834
NM     3156
Name: BUYER_STATE, dtype: int64

In [22]:
# test for duplication
assert not grouped1.duplicated(["BUYER_STATE", "BUYER_COUNTY", "year", "month"]).any()

In [28]:
grouped1.to_parquet("/Users/yangshining/Desktop/2021 fall @duke/IDS 720/mid-sem project/data/opioid_parquet/tx_yymm_separate.parquet", engine='fastparquet')

grouped2.to_parquet("/Users/yangshining/Desktop/2021 fall @duke/IDS 720/mid-sem project/data/opioid_parquet/tx_yymm_integrated.parquet", engine='fastparquet')

In [29]:
group1 = pd.read_parquet("/Users/yangshining/Desktop/2021 fall @duke/IDS 720/mid-sem project/data/opioid_parquet/tx_yymm_separate.parquet", engine='fastparquet')

group1.head(10)

,BUYER_STATE,BUYER_COUNTY,year,month,MME
0,LA,ACADIA,2006,1,2.821391e+05
1,LA,ACADIA,2006,2,1.607831e+06
2,LA,ACADIA,2006,3,1.971931e+06
3,LA,ACADIA,2006,4,1.578223e+06
4,LA,ACADIA,2006,5,1.914167e+06
5,LA,ACADIA,2006,6,1.915281e+06
6,LA,ACADIA,2006,7,1.938286e+06
7,LA,ACADIA,2006,8,1.762836e+06
8,LA,ACADIA,2006,9,1.709912e+06
9,LA,ACADIA,2006,10,2.388362e+06


In [30]:
group2 = pd.read_parquet("/Users/yangshining/Desktop/2021 fall @duke/IDS 720/mid-sem project/data/opioid_parquet/tx_yymm_integrated.parquet", engine='fastparquet')

group2.head(10)

,BUYER_STATE,BUYER_COUNTY,year_month,year,month,MME
0,LA,ACADIA,2006-01,190570,95,2.821391e+05
1,LA,ACADIA,2006-02,876622,874,1.607831e+06
2,LA,ACADIA,2006-03,1015036,1518,1.971931e+06
3,LA,ACADIA,2006-04,906712,1808,1.578223e+06
4,LA,ACADIA,2006-05,972910,2425,1.914167e+06
5,LA,ACADIA,2006-06,1113330,3330,1.915281e+06
6,LA,ACADIA,2006-07,934796,3262,1.938286e+06
7,LA,ACADIA,2006-08,968898,3864,1.762836e+06
8,LA,ACADIA,2006-09,950844,4266,1.709912e+06
9,LA,ACADIA,2006-10,1237702,6170,2.388362e+06


## Fill in population data of La Salle, LA

In [2]:
# load population

population = pd.read_parquet("/Users/yangshining/Desktop/pds2021-opioids-pds6/10_modified_data/pop_final.parquet", engine='fastparquet')
population.sample(10)

,STATE,COUNTY,STNAME,CTYNAME,Year,Population,STATE1,COUNTY1,fips
2771,48,413,Texas,Schleicher County,2003,2958,48,413,48413
16409,13,115,Georgia,Floyd County,2008,95922,13,115,13115
5375,40,35,Oklahoma,Craig County,2004,14739,40,035,40035
1296,26,87,Michigan,Lapeer County,2003,90573,26,087,26087
38707,13,35,Georgia,Butts County,2015,23520,13,035,13035
9087,48,281,Texas,Lampasas County,2005,18906,48,281,48281
8975,48,57,Texas,Calhoun County,2005,20715,48,057,48057
13276,13,233,Georgia,Polk County,2007,40789,13,233,13233
27721,40,53,Oklahoma,Grant County,2011,4563,40,053,40053
28450,51,117,Virginia,Mecklenburg County,2011,32560,51,117,51117


In [3]:
# create rows for La salle, LA


# create a list of Series

listofSeries = [
    pd.Series(
        [22, 59, "Louisiana", "La Salle Parish", 2003, 14356, "22", "059", "22059"],
        index=population.columns,
    ),
    pd.Series(
        [22, 59, "Louisiana", "La Salle Parish", 2004, 14366, "22", "059", "22059"],
        index=population.columns,
    ),
    pd.Series(
        [22, 59, "Louisiana", "La Salle Parish", 2005, 14313, "22", "059", "22059"],
        index=population.columns,
    ),
    pd.Series(
        [22, 59, "Louisiana", "La Salle Parish", 2006, 14519, "22", "059", "22059"],
        index=population.columns,
    ),
    pd.Series(
        [22, 59, "Louisiana", "La Salle Parish", 2007, 14570, "22", "059", "22059"],
        index=population.columns,
    ),
    pd.Series(
        [22, 59, "Louisiana", "La Salle Parish", 2008, 14667, "22", "059", "22059"],
        index=population.columns,
    ),
    pd.Series(
        [22, 59, "Louisiana", "La Salle Parish", 2009, 14717, "22", "059", "22059"],
        index=population.columns,
    ),
    pd.Series(
        [22, 59, "Louisiana", "La Salle Parish", 2010, 14908, "22", "059", "22059"],
        index=population.columns,
    ),
    pd.Series(
        [22, 59, "Louisiana", "La Salle Parish", 2011, 14941, "22", "059", "22059"],
        index=population.columns,
    ),
    pd.Series(
        [22, 59, "Louisiana", "La Salle Parish", 2012, 14862, "22", "059", "22059"],
        index=population.columns,
    ),
    pd.Series(
        [22, 59, "Louisiana", "La Salle Parish", 2013, 14821, "22", "059", "22059"],
        index=population.columns,
    ),
    pd.Series(
        [22, 59, "Louisiana", "La Salle Parish", 2014, 14875, "22", "059", "22059"],
        index=population.columns,
    )
]


# append these rows to population dataframe
population = population.append(listofSeries, ignore_index=True)


# check if the rows are correctly appended
population[
    (population["STNAME"] == "Louisiana")
    & (population["CTYNAME"] == "La Salle Parish")
]


,STATE,COUNTY,STNAME,CTYNAME,Year,Population,STATE1,COUNTY1,fips
41483,22,59,Louisiana,La Salle Parish,2003,14356,22,059,22059
41484,22,59,Louisiana,La Salle Parish,2004,14366,22,059,22059
41485,22,59,Louisiana,La Salle Parish,2005,14313,22,059,22059
41486,22,59,Louisiana,La Salle Parish,2006,14519,22,059,22059
41487,22,59,Louisiana,La Salle Parish,2007,14570,22,059,22059
41488,22,59,Louisiana,La Salle Parish,2008,14667,22,059,22059
41489,22,59,Louisiana,La Salle Parish,2009,14717,22,059,22059
41490,22,59,Louisiana,La Salle Parish,2010,14908,22,059,22059
41491,22,59,Louisiana,La Salle Parish,2011,14941,22,059,22059
41492,22,59,Louisiana,La Salle Parish,2012,14862,22,059,22059


In [4]:
# Unify the data type of some columns 

population["STATE"] = population["STATE"].astype(int)
population["COUNTY"] = population["COUNTY"].astype(int)
population["Year"] = population["Year"].astype(int)


In [5]:
# store the data

population.to_parquet("/Users/yangshining/Desktop/pds2021-opioids-pds6/10_modified_data/pop_add_lasalle.parquet", engine='fastparquet')

## Merge with FIPS

In [6]:
# load opioid data of Texas and control

opi_tx = pd.read_parquet("/Users/yangshining/Desktop/2021 fall @duke/IDS 720/mid-sem project/data/opioid_parquet/tx_yymm_separate.parquet", engine='fastparquet')

opi_tx.sample(10)

,BUYER_STATE,BUYER_COUNTY,year,month,MME
37475,TX,SAN PATRICIO,2010,6,1.581434e+06
8641,NM,QUAY,2010,7,2.953094e+05
10296,OK,ATOKA,2013,8,5.561003e+05
21983,TX,CHILDRESS,2009,6,2.674555e+05
38027,TX,SHERMAN,2012,12,4.615650e+04
11207,OK,CHOCTAW,2008,8,3.593783e+05
24129,TX,DIMMIT,2008,2,9.366810e+04
35897,TX,POLK,2009,7,1.044961e+06
11241,OK,CHOCTAW,2011,6,7.672155e+05
3132,LA,LAFOURCHE,2012,5,2.589456e+06


In [8]:
# test for duplication

assert not opi_tx.duplicated(["BUYER_STATE", "BUYER_COUNTY", "year", "month"]).any()

In [9]:
# Load fips data
fips = pd.read_csv("https://raw.githubusercontent.com/kjhealy/fips-codes/master/county_fips_master.csv", encoding="ISO-8859-1")

# Modify the county name in fips for better merging with the opioid data
fips['county_name']= fips['county_name'].str[:-7]
fips['county_name']= fips['county_name'].str.upper()

fips.head()

,fips,county_name,state_abbr,state_name,long_name,sumlev,region,division,state,county,crosswalk,region_name,division_name
0,1001,AUTAUGA,AL,Alabama,Autauga County AL,50.0,3.0,6.0,1.0,1.0,3-6-1-1,South,East South Central
1,1003,BALDWIN,AL,Alabama,Baldwin County AL,50.0,3.0,6.0,1.0,3.0,3-6-1-3,South,East South Central
2,1005,BARBOUR,AL,Alabama,Barbour County AL,50.0,3.0,6.0,1.0,5.0,3-6-1-5,South,East South Central
3,1007,BIBB,AL,Alabama,Bibb County AL,50.0,3.0,6.0,1.0,7.0,3-6-1-7,South,East South Central
4,1009,BLOUNT,AL,Alabama,Blount County AL,50.0,3.0,6.0,1.0,9.0,3-6-1-9,South,East South Central


In [10]:
# This cell is to fix the county name inconsistency between the two datasets

# Fix ST JOHN THE BAPTIST
opi_tx['BUYER_COUNTY'] = np.where(opi_tx["BUYER_COUNTY"].str[:3] == "ST ", "ST. " + opi_tx["BUYER_COUNTY"].str[3:] , opi_tx["BUYER_COUNTY"])

# Fix SAINT

opi_tx['BUYER_COUNTY'] = np.where(opi_tx["BUYER_COUNTY"].str[:6] == "SAINT ", "ST. " + opi_tx["BUYER_COUNTY"].str[6:] , opi_tx["BUYER_COUNTY"])

#Fix DONA ANA

fips['county_name'] = np.where(fips['county_name'] == "DOÐA ANA", "DONA ANA" , fips['county_name'])

# Fix DE SOTO
opi_tx['BUYER_COUNTY'] = np.where(opi_tx["BUYER_COUNTY"] == "DE SOTO", "DESOTO" , opi_tx["BUYER_COUNTY"])
fips['county_name'] = np.where(fips['county_name'] == "DE SOTO", "DESOTO" , fips['county_name'])

# Fix DE KALB
opi_tx['BUYER_COUNTY'] = np.where(opi_tx["BUYER_COUNTY"] == "DE KALB", "DEKALB" , opi_tx["BUYER_COUNTY"])

# Fix DE WITT
opi_tx['BUYER_COUNTY'] = np.where(opi_tx["BUYER_COUNTY"] == "DE WITT", "DEWITT" , opi_tx["BUYER_COUNTY"])


In [11]:
# merging

# Subset the fips data - keep only the columns we need 
fips_sub = fips[["county_name", "state_abbr", "fips"]]

opi_merge = pd.merge(fips_sub, opi_tx, how = "right",left_on = ["state_abbr", "county_name"], right_on=["BUYER_STATE", "BUYER_COUNTY"])

# Test if there's any na values
assert len(opi_merge[opi_merge["fips"].isna()]) ==0

In [12]:
# load population data after appending the rows of La Salle, Louisiana

population = pd.read_parquet("/Users/yangshining/Desktop/pds2021-opioids-pds6/10_modified_data/pop_add_lasalle.parquet", engine='fastparquet')

In [13]:
population

,STATE,COUNTY,STNAME,CTYNAME,Year,Population,STATE1,COUNTY1,fips
0,1,0,Alabama,Alabama,2003,4503491,01,000,01000
1,1,1,Alabama,Autauga County,2003,46800,01,001,01001
2,1,3,Alabama,Baldwin County,2003,151509,01,003,01003
3,1,5,Alabama,Barbour County,2003,28594,01,005,01005
4,1,7,Alabama,Bibb County,2003,21399,01,007,01007
...,...,...,...,...,...,...,...,...,...
41490,22,59,Louisiana,La Salle Parish,2010,14908,22,059,22059
41491,22,59,Louisiana,La Salle Parish,2011,14941,22,059,22059
41492,22,59,Louisiana,La Salle Parish,2012,14862,22,059,22059
41493,22,59,Louisiana,La Salle Parish,2013,14821,22,059,22059


In [14]:
# Change the population FIPS code to strings
population["fips"] = population["fips"].astype(str)

# Modify data types of columns - prepare for merging

population["Year"] = population["Year"].astype(int)

opi_merge["fips"] = opi_merge["fips"].astype(str)

opi_merge["fips"] = opi_merge["fips"].apply(lambda x: x.zfill(5)) # fill fips code up to 5 digits

opi_tx_pop = pd.merge(opi_merge, population, left_on=["fips", "year"], right_on=["fips", "Year"], how='left') #Merging based on opioid dataset



# Check if there's any missing value in population

opi_tx_pop[opi_tx_pop["Population"].isnull()]  # La Salle is NOT considered to be a county in Louisiana

,county_name,state_abbr,fips,BUYER_STATE,BUYER_COUNTY,year,month,MME,STATE,COUNTY,STNAME,CTYNAME,Year,Population,STATE1,COUNTY1


In [15]:
# store the data

opi_tx_pop.to_parquet("/Users/yangshining/Desktop/pds2021-opioids-pds6/10_modified_data/opi_tx_pop.parquet", engine='fastparquet')